**Clean Data**

In [27]:
import pandas as pd


df = pd.read_csv('carData.csv')
df.loc[df['fuel_type'].isnull() & df['engine'].str.contains('Electric Motor', case=False, na=False), 'fuel_type'] = 'Electric'

df.loc[df['fuel_type'].isnull()].head()
df.loc[df['fuel_type'].isnull() & (df['engine'].str.lower() == 'electric'), 'fuel_type'] = 'Electric'
df.loc[df['fuel_type'].isnull() & (df['brand'].str.lower() == 'tesla'), 'fuel_type'] = 'Electric'
df.loc[df['fuel_type'].isnull() & df['engine'].str.contains('battery', case=False, na=False), 'fuel_type'] = 'Electric'

df['mileage'] = df['mileage'].str.replace(r'[^\d]', '', regex=True).astype(int)
df['price'] = df['price'].replace(r'[$,]', '', regex=True).astype(int)

df['fuel_type'] = df['fuel_type'].replace('Plug-In Hybrid', 'Hybrid')
df['clean_title'] = df['clean_title'].fillna('No')
df['accident'] = df['accident'].fillna('Unknown')
df.to_csv('cleaned_car_data.csv',index=False)

df = pd.read_csv('cleaned_car_data.csv')
df['model'].unique().shape[0]


# df.head()

1898

In [28]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('cleaned_car_data.csv')

import plotly.graph_objects as go

fig = px.histogram(df, x='price', nbins=80)

# Add a vertical line at $250,000 to show cutoff for “regular cars”
fig.add_vline(
    x=250000,
    line_width=3,
    line_dash="dash",
    line_color="red",
    annotation_text="Supercar/Outlier Threshold ($250k)",
    annotation_position="top right"
)

fig.update_layout(
    title='Car Price Distribution',
    xaxis_title='Price ($)',
    yaxis_title='Count',
    xaxis=dict(tick0=0, dtick=250000)
)

fig.show()



In [29]:
import pandas as pd
import re
import numpy as np

df = pd.read_csv('cleaned_car_data.csv')



df.head()



def extract_hp(engine_str):
    match = re.search(r'(\d+(?:\.\d+)?)\s*hp', str(engine_str).lower())  # handles float like 211.0
    if match:
        return float(match.group(1))
    return np.nan

df['HP'] = df['engine'].apply(extract_hp)

df['HP'] = df['engine'].apply(extract_hp)

df['HP'].head()
df.to_csv('cleaned_car_data.csv')
unique_counts = df.nunique().reset_index()
unique_counts.columns = ['Column', 'Unique Values']
unique_counts


,Column,Unique Values
0,brand,57
1,model,1898
2,model_year,34
3,milage,2818
4,fuel_type,7
5,engine,1146
6,transmission,62
7,ext_col,319
8,int_col,156
9,accident,3


**Split Data**

In [30]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# 1. Load the dataset
df = pd.read_csv('cleaned_car_data.csv')
df = df[df['price'] < 250000]  # Remove extreme outliers

# 2. Convert 'clean_title' to binary
df['clean_title'] = df['clean_title'].map({'Yes': 1, 'No': 0}).astype(int)

# 3. Target encode high-cardinality text features
df['brand_encoded'] = df.groupby('brand')['price'].transform('mean')
df['model_encoded'] = df.groupby('model')['price'].transform('mean')
df['engine_encoded'] = df.groupby('engine')['price'].transform('mean')
df['transmission_encoded'] = df.groupby('transmission')['price'].transform('mean')

# 4. Extract HP from engine string
def extract_hp(engine_str):
    match = re.search(r'(\d+(?:\.\d+)?)\s*hp', str(engine_str).lower())
    if match:
        return float(match.group(1))
    return np.nan

df['HP'] = df['engine'].apply(extract_hp)
df['HP'].fillna(df['HP'].median(), inplace=True)  # Fill missing HP values with median

# 5. Bin 'mileage' to reduce noise
df['mileage_binned'] = pd.qcut(df['mileage'], q=10, labels=False)

# 6. Define feature groups
numerical_features = ['model_year', 'mileage_binned', 'HP']  # Added HP here
binary_features = ['clean_title']
target_encoded_features = ['brand_encoded', 'model_encoded', 'engine_encoded', 'transmission_encoded']
categorical_features = ['fuel_type', 'ext_col', 'int_col', 'accident']

# 7. One-hot encode remaining low-cardinality categorical features
df_encoded = pd.get_dummies(df[categorical_features], drop_first=True)

# 8. Combine features into X
X = pd.concat([df[numerical_features + binary_features + target_encoded_features], df_encoded], axis=1)
y = df['price']

# 9. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 10. Train RandomForestRegressor
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=0,
    n_jobs=-1
)
model.fit(X_train, y_train)

# 11. Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error: {np.sqrt(mse):.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R^2 Score: {r2:.4f}")

# 12. Feature importances
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
feature_importance_df.to_csv('feature_importance.csv', index=False)

# 13. Save predictions with errors
X_test_copy = X_test.copy()
X_test_copy['predicted_price'] = y_pred
X_test_copy['actual_price'] = y_test.values
X_test_copy['pct_error'] = ((X_test_copy['predicted_price'] - X_test_copy['actual_price']) / X_test_copy['actual_price']) * 100
X_test_copy['abs_pct_error'] = X_test_copy['pct_error'].abs()
X_test_copy.to_csv('test_predictions_with_errors.csv', index=False)


Root Mean Squared Error: 9207.52
Mean Absolute Error: 4792.89
R^2 Score: 0.9308


In [31]:
# import plotly.express as px
# import pandas as pd
# # Create a DataFrame for comparison
# results_df = pd.DataFrame({
#     'Actual Price': y_test,
#     'Predicted Price': y_pred,
#     'Brand': df.loc[y_test.index, 'brand']
# })


# fig = px.scatter(
#     results_df,
#     x='Actual Price',
#     y='Predicted Price',
#     title='Actual vs Predicted Car Prices',
#     labels={'Actual Price': 'Actual Price ($)', 'Predicted Price': 'Predicted Price ($)'},
#     opacity=0.6,
#     trendline="ols",
#     hover_name='Brand'
# )


# fig.add_shape(
#     type="line",
#     x0=results_df['Actual Price'].min(), y0=results_df['Actual Price'].min(),
#     x1=results_df['Actual Price'].max(), y1=results_df['Actual Price'].max(),
#     line=dict(color="red", dash="dash"),
#     name='Perfect Prediction'
# )


# fig.show()

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Create a DataFrame for comparison
results_df = pd.DataFrame({
    'Actual Price': y_test,
    'Predicted Price': y_pred,
    'Brand': df.loc[y_test.index, 'brand']
})

# Base scatter plot with trendline
fig = px.scatter(
    results_df,
    x='Actual Price',
    y='Predicted Price',
    title='Actual vs Predicted Car Prices',
    labels={'Actual Price': 'Actual Price ($)', 'Predicted Price': 'Predicted Price ($)'},
    opacity=0.6,
    hover_name='Brand',
    trendline='ols'
)

# Add perfect prediction line (y = x) with legend
fig.add_trace(go.Scatter(
    x=[results_df['Actual Price'].min(), results_df['Actual Price'].max()],
    y=[results_df['Actual Price'].min(), results_df['Actual Price'].max()],
    mode='lines',
    line=dict(color='red', dash='dash'),
    name='Perfect Prediction'
))

fig.update_layout(legend_title_text='Legend')
fig.show()


In [34]:
import plotly.graph_objects as go
import statsmodels.api as sm
import pandas as pd

# Prepare data
results_df = pd.DataFrame({
    'Actual Price': y_test,
    'Predicted Price': y_pred,
    'Brand': df.loc[y_test.index, 'brand']
})

# OLS regression line (Predicted vs Actual)
X = sm.add_constant(results_df['Actual Price'])
ols_model = sm.OLS(results_df['Predicted Price'], X).fit()
results_df['OLS Prediction'] = ols_model.predict(X)

# 1. Scatter plot of actual vs predicted
scatter = go.Scatter(
    x=results_df['Actual Price'],
    y=results_df['Predicted Price'],
    mode='markers',
    name='Used-Car',
    marker=dict(color='rgba(0, 100, 255, 0.5)', size=5)
)

# 2. Perfect prediction line (y = x)
line_identity = go.Scatter(
    x=[results_df['Actual Price'].min(), results_df['Actual Price'].max()],
    y=[results_df['Actual Price'].min(), results_df['Actual Price'].max()],
    mode='lines',
    name='Perfect Prediction',
    line=dict(color='green', dash='dash')
)

# 3. OLS Regression Line
line_ols = go.Scatter(
    x=results_df['Actual Price'],
    y=results_df['OLS Prediction'],
    mode='lines',
    name='OLS Regression Line',
    line=dict(color='red')
)

# 4. Build the figure
fig = go.Figure(data=[scatter, line_identity, line_ols])
fig.update_layout(
    title='Predicted vs Actual Car Prices',
    xaxis_title='Actual Price ($)',
    yaxis_title='Predicted Price ($)',
    legend_title='Legend',
    width=900,
    height=600
)

fig.show()


In [35]:
import plotly.express as px

# Ensure brand column exists in X_test_copy
X_test_copy['brand'] = df.loc[X_test_copy.index, 'brand']

# 1. Compute percentage error
X_test_copy['pct_error'] = ((X_test_copy['predicted_price'] - X_test_copy['actual_price']) / X_test_copy['actual_price']) * 100

# 2. Compute average percent error per brand
avg_pct_error_df = X_test_copy.groupby('brand', as_index=False)['pct_error'].mean()
avg_pct_error_df.rename(columns={'pct_error': 'Avg % Error'}, inplace=True)

# 3. Sort by average percent error
avg_pct_error_df = avg_pct_error_df.sort_values(by='Avg % Error')

# 4. Plot
fig = px.bar(
    avg_pct_error_df,
    x='brand',
    y='Avg % Error',
    color='brand',
    title='Average Percent Error by Brand',
    labels={'Avg % Error': 'Average % Error', 'brand': 'Brand'},
)
fig.update_layout(xaxis_tickangle=-45, showlegend=False)
fig.show()
